## Cement Micro-localization Model - WP 2.1a (Phase 1)

This notebook is a modified version of the model build completed for WP 2.1a in Phase 1 of the SFI/ALD project. It has been modified to run within EarthAI Notebook. The only change is that instead of reading image chips from a Google Drive folder, it copies them from the shared SFI/ALD AWS S3 bucket to fast local storage in EarthAI Notebook for training the deep learning model.

Input chips for training deep learning model:
* 1024x1024x3 RGB images for training in jpeg format: s3://sfi-shared-assets/cement-microloc-phase1/images/data/
* 1024x1024 binary masks in jpeg format: s3://sfi-shared-assets/cement-microloc-phase1/CIFF-ALD/masks/data/
* 1024x1024x3 RGB images for testing in jpeg format: s3://sfi-shared-assets/cement-microloc-phase1/CIFF-ALD/test/

Outputs:
* Model: s3://sfi-shared-assets/cement-microloc-phase1/model/cement_model.h5
* Plots: s3://sfi-shared-assets/cement-microloc-phase1/plots/

Attribution for original source code:
Steven Reece (reece@robots.ox.ac.uk)
Version 1.0

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import glob
from tensorflow.keras.backend import epsilon, clip
from tensorflow.keras import layers, models
#from google.colab import drive
from sklearn.metrics import confusion_matrix
from keras.losses import binary_crossentropy
import os
import boto3
import re

#drive.mount('/content/drive/')

%matplotlib inline

In [ ]:
NBANDS = 3 # RGB imagery
IMG_WIDTH = 1024
IMG_HEIGHT = 1024
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, NBANDS)

# Encoder
vgg = tf.keras.applications.VGG16(input_shape=INPUT_SHAPE, weights='imagenet',include_top=False)
vgg.trainable = False

c1 = vgg.get_layer('block1_conv2').output
c2 = vgg.get_layer('block2_conv2').output
c3 = vgg.get_layer('block3_conv2').output
c4 = vgg.get_layer('block4_conv2').output
cv = vgg.get_layer('block5_conv2').output

# Decoder
u5 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(cv)
u5 = tf.keras.layers.concatenate([u5, c4])
c5 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u5)
c5 = tf.keras.layers.Dropout(0.2)(c5)
c5 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c3])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c2])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c1])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

output = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c8)

model = models.Model(vgg.input,output)

model.summary()

In [ ]:
# Define a weighted categorical cross-entropy cost function

def custom_accuracy_p(y_true,y_pred):
# True positive rate
  sh=tf.shape(y_pred)
  nvals=tf.size(y_pred)

  y_pred = tf.reshape(y_pred, [nvals,1])
  y_true = tf.reshape(y_true, [nvals,1])

  n_y_true = tf.reduce_sum(y_true)
  
  return tf.divide(tf.reduce_sum(tf.multiply(y_true,y_pred)),tf.maximum(epsilon(),n_y_true))

def custom_accuracy_n(y_true,y_pred):
# True negative rate
  sh=tf.shape(y_pred)
  nvals=tf.size(y_pred)

  y_pred = tf.reshape(y_pred, [nvals,1])
  y_true = tf.reshape(y_true, [nvals,1])
  
  n_y_false = tf.reduce_sum(1.0-y_true)

  return tf.divide(tf.reduce_sum(tf.multiply(1.0-y_true[:,0],1.0-y_pred[:,0])),tf.maximum(epsilon(),n_y_false))

def custom_loss(y_true,y_pred):
# Weighted binary cross entropy loss
  sh=tf.shape(y_pred)
  nvals=tf.size(y_pred)

  y_pred = tf.reshape(y_pred, [nvals,1])
  y_true = tf.reshape(y_true, [nvals,1])

  y_log_pred_t = tf.math.log(clip(y_pred, epsilon(), 9999.0))
  y_log_pred_f = tf.math.log(clip(1.0-y_pred, epsilon(), 9999.0))
  #n_y_true = tf.divide(tf.reduce_sum(y_true),tf.cast(nvals,tf.float32))
  #n_y_false = 1.0 - n_y_true

  n_y_false = 2.0 # Add weight to plant classes.
  n_y_true = 1.0

  return -tf.divide(tf.reduce_sum(n_y_false * y_log_pred_t * y_true + n_y_true * y_log_pred_f * (1.0-y_true)),tf.cast(nvals,tf.float32))

In [ ]:
model.compile(optimizer='adam',loss=custom_loss, metrics=[custom_accuracy_p,custom_accuracy_n])

In [ ]:
# Install AWS CLI
! pip install awscli

In [ ]:
# Copy image, masks, and test chips from S3
# Note that the /scratch directory is the fast local storage in EarthAI Notebook - best place to temporarily store
# image chips for deep learning; these data do NOT persist between sessions
! aws s3 cp s3://sfi-shared-assets/cement-microloc-phase1 /scratch/cement-microloc-phase1 --recursive --quiet

In [ ]:
BATCH_SIZE = 6 # Limitations of Colab (free license).
USE_STORED_MODEL = False # Change to True to use saved model.

training_images = r'/scratch/cement-microloc-phase1/images'
training_masks = r'/scratch/cement-microloc-phase1/masks'

if not 'model' in os.listdir('/scratch/cement-microloc-phase1'):
    os.mkdir(r'/scratch/cement-microloc-phase1/model')
model_path = r'/scratch/cement-microloc-phase1/model/cement_model.h5'

callbacks = [
        tf.keras.callbacks.ModelCheckpoint(model_path,
                                           monitor='val_loss',
                                           save_best_only=True,
                                           save_weights_only=True,
                                           verbose=1),
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='loss')]

# generators for data augmentation -------
seed = 1
images_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    zoom_range = 0.2,
    horizontal_flip=True,
    fill_mode='reflect',
    validation_split=0.2)

masks_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    zoom_range = 0.2,
    horizontal_flip=True,
    fill_mode='reflect',
    validation_split=0.2)

image_generator = images_datagen.flow_from_directory(
    training_images,
    target_size=(IMG_WIDTH,IMG_HEIGHT),
    class_mode=None,
    batch_size = BATCH_SIZE,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    training_masks,
    target_size=(IMG_WIDTH,IMG_HEIGHT),
    class_mode=None,
    batch_size = BATCH_SIZE,
    color_mode = 'grayscale',
    seed=seed)

val_image_generator = images_datagen.flow_from_directory(
    training_images,
    target_size=(IMG_WIDTH,IMG_HEIGHT),
    class_mode=None,
    batch_size = BATCH_SIZE,
    subset='validation',
    seed=seed)

val_mask_generator = masks_datagen.flow_from_directory(
    training_masks,
    target_size=(IMG_WIDTH,IMG_HEIGHT),
    class_mode=None,
    batch_size = BATCH_SIZE,
    color_mode = 'grayscale',
    subset='validation',
    seed=seed)

train_generator = zip(image_generator, mask_generator)
validation_generator = zip(val_image_generator, val_mask_generator)

if USE_STORED_MODEL:
  model.load_weights(model_path)
else:
  model.fit(train_generator, validation_data=validation_generator, 
            steps_per_epoch=len(training_images)//BATCH_SIZE, 
            validation_steps=len(training_images)//BATCH_SIZE,
            epochs=50, callbacks=callbacks)

In [ ]:
# Confusion matrix pretty print

def print_cm(cm, labels):
  print()
  print("    t/p", end=" ")
    
  for label in labels:
    print("%{0}s".format(26) % label, end=" ")
    print("        ", end=" ") 
  print()
  # Print rows
  for i, label1 in enumerate(labels):
    print("%{0}s".format(7) % label1, end=" ")
    for j in range(len(labels)):
      frac = 100.0*cm[i,j]/np.sum(cm[i,:])
      print("%{0}.d   ".format(20) % cm[i,j], end=" ")
      print("(%{0}.2f%%)".format(6) % frac, end=" ")
    print()

  print()

In [ ]:
all_aoi_files = glob.glob(r'/scratch/cement-microloc-phase1/test/CHN*.jpg')

if not 'plots' in os.listdir('/scratch/cement-microloc-phase1'):
    os.mkdir(r'/scratch/cement-microloc-phase1/plots')
for filen in range(len(all_aoi_files)):
  aoi_file=all_aoi_files[filen]
  mask_file = str.replace(aoi_file,'test/CHN','test/mask_CHN')

  print(aoi_file)

  aoi = plt.imread(aoi_file)
  aoi = aoi/255
  mask = plt.imread(mask_file)
  mask = mask/255
  preds_test = np.squeeze(model.predict(np.expand_dims(aoi.astype(float),axis = 0)))

  f = plt.figure(figsize=[60,60])
  f.add_subplot(3,3,1) 
  plt.imshow(aoi)
  plt.title('Chip',fontsize = 80)
  f.add_subplot(3,3,2) 
  plt.imshow(mask, cmap='gray')
  plt.title('Mask',fontsize = 80)
  f.add_subplot(3,3,3) 
  plt.imshow(preds_test, cmap='gray')
  for i in range(7):
    f.add_subplot(3,3,i+3) 
    pr = 0.1*(i+2)
    preds_test_t = (preds_test > pr).astype(np.uint8)
    plt.imshow(np.squeeze(preds_test_t), cmap='gray')
    plt.title('pr(plant) > ' + str(np.round(pr,1)),fontsize = 80)
    print_cm(confusion_matrix(np.uint8(mask.flatten()),
                              preds_test_t.flatten(),
                              labels=[0,1]),
             ['backg','plant'])
 # plt.show()

  out_file=r'/scratch/cement-microloc-phase1/plots/plot_' + str(filen) + '.jpg'
  plt.savefig(out_file)
  plt.close()

In [ ]:
# Store model in S3
s3 = boto3.resource('s3')
bucket = s3.Bucket('sfi-shared-assets')

bucket.upload_file(r'/scratch/cement-microloc-phase1/model/cement_model.h5', \
                   'cement-microloc-phase1/model/cement_model.h5')

In [ ]:
# Store generated plots in S3
plots_list = os.listdir(r'/scratch/cement-microloc-phase1/plots')

# Upload plot jpgs to s3
for img in plots_list:
    if re.search("CHN", img):
        bucket.upload_file(r'/scratch/cement-microloc-phase1/plots/'+img, \
                           'cement-microloc-phase1/plots/'+img)